<a href="https://colab.research.google.com/github/jkf87/seg-diary/blob/main/handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers
!pip install pandas
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# 데이터 로드
df = pd.read_csv('diary_emotions.csv')

# 데이터 구조 확인
print("데이터 크기:", df.shape)
print("\n컬럼명:", df.columns.tolist())
print("\n처음 5개 행:")
display(df.head())

# 레이블 분포 확인
print("\n감정 레이블 분포:")
display(df['label'].value_counts())

데이터 크기: (1000, 2)

컬럼명: ['text', 'label']

처음 5개 행:


,text,label
0,지금 기분이 편리했다.,1
1,내 마음이 허전했다.,0
2,하루종일 다정했다.,1
3,전체적으로 행복감이 들었다.,1
4,나는 빛났다.,1



감정 레이블 분포:


,count
label,
1,500
0,500


In [ ]:
class DiaryAnalyzer:
    def __init__(self):
        # ko-sroberta-multitask 모델 로드
        self.model = SentenceTransformer('jhgan/ko-sroberta-multitask')

        # diary_emotions.csv 데이터 로드
        self.df = pd.read_csv('diary_emotions.csv')

        # 컬럼명 설정
        self.text_column = 'text'
        self.emotion_column = 'label'

        # 문장 임베딩 계산 및 저장
        self.embeddings = self.model.encode(self.df[self.text_column].tolist(),
                                          convert_to_tensor=True)

    def analyze_sentiment(self, text):
        if not text.strip():
            return {"긍정": "0%", "부정": "0%"}

        # 입력 텍스트의 임베딩 계산
        input_embedding = self.model.encode(text, convert_to_tensor=True)

        # 코사인 유사도 계산
        similarities = cosine_similarity(
            input_embedding.cpu().numpy().reshape(1, -1),
            self.embeddings.cpu().numpy()
        )

        # 가장 유사한 상위 5개 문장 찾기
        top_5_indices = similarities[0].argsort()[-5:][::-1]

        # 감정 점수 계산
        positive_score = 0
        negative_score = 0

        for idx in top_5_indices:
            similarity = similarities[0][idx]
            if self.df.iloc[idx][self.emotion_column] == 1:
                positive_score += similarity
            else:
                negative_score += similarity

        # 정규화
        total = positive_score + negative_score
        if total == 0:
            return {"긍정": "50%", "부정": "50%"}

        positive_ratio = positive_score / total
        negative_ratio = negative_score / total

        # 유사한 문장 정보 포함
        similar_texts = []
        for idx in top_5_indices:
            emotion = "긍정" if self.df.iloc[idx][self.emotion_column] == 1 else "부정"
            similar_texts.append({
                "text": self.df.iloc[idx][self.text_column],
                "emotion": emotion,
                "similarity": f"{similarities[0][idx]:.3f}"
            })

        return {
            "긍정": f"{positive_ratio:.1%}",
            "부정": f"{negative_ratio:.1%}",
            "similar_texts": similar_texts
        }

In [ ]:
# 분석기 초기화
analyzer = DiaryAnalyzer()

# 예시 일기
example_diaries = {
    "긍정적인 일기": """오늘은 정말 행복한 하루였다. 아침에 일찍 일어나서 상쾌하게 운동을 했고,
회사에서는 내가 진행하던 프로젝트가 성공적으로 마무리되어서 팀장님께 칭찬을 받았다.
퇴근 후에는 오랜만에 만난 대학 친구들과 맛있는 저녁을 먹으며 즐거운 시간을 보냈다.
특히 친구가 내 조언 덕분에 좋은 결과를 얻었다고 고마워해서 더욱 뿌듯했다.
이런 날들이 계속되었으면 좋겠다.""",

    "중립적인 일기": """오늘은 아침부터 기분이 좋았다. 날씨도 화창하고 산책하기 좋아서 공원에 다녀왔다.
하지만 점심에 실수로 커피를 쏟아서 옷이 더러워졌고, 중요한 미팅 직전이라 당황스러웠다.
다행히 동료가 여분의 옷을 빌려줘서 무사히 미팅을 마쳤다.
저녁에는 좋아하는 드라마를 보면서 휴식을 취했지만, 내일 있을 발표가 걱정되어 마음이 좀 무거웠다.""",

    "부정적인 일기": """정말 최악의 하루였다. 아침부터 알람이 울리지 않아 지각을 했고,
급하게 준비한 프레젠테이션에서는 실수를 연발했다. 점심시간에는 커피를 쏟아서
옷이 엉망이 되었고, 오후 회의에서는 상사에게 심하게 질책을 받았다.
퇴근길에 비를 맞아 감기 기운도 있는 것 같다. 요즘 계속 안 좋은 일만 생기는 것 같아
우울하다."""
}

# 각 예시 일기 분석
for diary_type, diary_text in example_diaries.items():
    print(f"\n=== {diary_type} 분석 결과 ===")
    result = analyzer.analyze_sentiment(diary_text)
    print(f"긍정: {result['긍정']}")
    print(f"부정: {result['부정']}")
    print("\n유사한 일기:")
    for i, similar in enumerate(result['similar_texts'], 1):
        print(f"\n{i}. [{similar['emotion']}] (유사도: {similar['similarity']})")
        print(similar['text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


=== 긍정적인 일기 분석 결과 ===
긍정: 100.0%
부정: 0.0%

유사한 일기:

1. [긍정] (유사도: 0.715)
오늘 하루는 보람 있었다.

2. [긍정] (유사도: 0.700)
오늘 하루는 즐거운 기분이었다.

3. [긍정] (유사도: 0.698)
오늘은 정말 기쁨을 느꼈다.

4. [긍정] (유사도: 0.694)
오늘은 정말 즐거웠다.

5. [긍정] (유사도: 0.693)
오늘은 정말 성공적인 하루였다.

=== 중립적인 일기 분석 결과 ===
긍정: 59.9%
부정: 40.1%

유사한 일기:

1. [긍정] (유사도: 0.582)
오늘 하루는 즐거운 기분이었다.

2. [부정] (유사도: 0.582)
오늘 하루는 걱정스러웠다.

3. [부정] (유사도: 0.582)
오늘 하루는 걱정스러웠다.

4. [긍정] (유사도: 0.579)
지금 기분이 멋진 하루였다.

5. [긍정] (유사도: 0.575)
내 마음이 상쾌했다.

=== 부정적인 일기 분석 결과 ===
긍정: 0.0%
부정: 100.0%

유사한 일기:

1. [부정] (유사도: 0.681)
하루종일 실망감이 들었다.

2. [부정] (유사도: 0.672)
하루종일 짜증났다.

3. [부정] (유사도: 0.660)
하루종일 불만족스러웠다.

4. [부정] (유사도: 0.660)
하루종일 불만족스러웠다.

5. [부정] (유사도: 0.654)
오늘 하루는 낙담했다.


In [ ]:
def create_interface():
    def process_diary(text):
        result = analyzer.analyze_sentiment(text)
        output = f"긍정: {result['긍정']}\n부정: {result['부정']}\n\n"
        output += "가장 유사한 일기들:\n"
        for i, similar in enumerate(result['similar_texts'], 1):
            output += f"{i}. [{similar['emotion']}] (유사도: {similar['similarity']})\n"
            output += f"{similar['text']}\n\n"
        return output

    interface = gr.Interface(
        fn=process_diary,
        inputs=gr.Textbox(lines=5, label="오늘의 일기를 작성해주세요"),
        outputs=gr.Textbox(label="감정 분석 결과"),
        title="일기 감정 분석기",
        description="당신의 일기에 담긴 감정을 분석해드립니다.",
        examples=[
            [example_diaries["긍정적인 일기"]],
            [example_diaries["중립적인 일기"]],
            [example_diaries["부정적인 일기"]]
        ]
    )

    return interface

interface = create_interface()
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6cd2c521a12447075f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
